In [1]:
from src.extracter import Extracter

In [2]:
import glob
import os
import src.tools.myconfig as config
import shutil
import subprocess

In [3]:
extracter = Extracter()

In [4]:
for pdf_file in glob.glob(os.path.join(config.PDF_DIR, '*.pdf')):
    csv_file = os.path.join(
        config.CSV_DIR, 
        '{}.csv'.format(os.path.basename(pdf_file)[:-4])
    )
    if not (os.path.isfile(csv_file)):
        extracter.tabula(pdf_file, csv_file)

In [5]:
empty_csv_files = []

for csv_file in sorted(glob.glob(os.path.join(config.CSV_DIR, '*.csv'))):
    if os.path.getsize(csv_file) == 0:
        empty_csv_files.append(csv_file)

TOTAL : 260
DIRTY : 182


In [11]:
if not (os.path.isdir(config.ERROR_PDF_DIR)):
    os.mkdir(config.ERROR_PDF_DIR)
if not (os.path.isdir(config.OCR_PDF_DIR)):
    os.mkdir(config.OCR_PDF_DIR)
if not (os.path.isdir(config.OCR_CSV_DIR)):
    os.mkdir(config.OCR_CSV_DIR)

for csv_file in empty_csv_files:
    filename = os.path.basename(csv_file)[:-4]
    pdf_file = os.path.join(config.PDF_DIR, '{}.pdf'.format(filename))
    old_ocr_pdf_file = os.path.join(config.PDF_DIR, '{}_ocr.pdf'.format(filename))
    ocr_pdf_file = os.path.join(config.OCR_PDF_DIR, '{}.pdf'.format(filename))
    ocr_csv_file = os.path.join(config.OCR_CSV_DIR, '{}.csv'.format(filename))
    
    if os.path.isfile(ocr_csv_file):
        continue
    
    try:
        subprocess.check_call(['pypdfocr', '{}'.format(pdf_file)])
    except subprocess.CalledProcessError:
        shutil.copy2(
            pdf_file, 
            os.path.join(config.ERROR_PDF_DIR, '{}.pdf'.format(filename))
        )
        continue
    shutil.move(old_ocr_pdf_file, ocr_pdf_file)
    if not (os.path.isfile(ocr_csv_file)):
        extracter.tabula(ocr_pdf_file, ocr_csv_file)

In [ ]:
csv_files_list = glob.glob(os.path.join(config.CSV_DIR, '*.csv'))
print('TOTAL : {}'.format(len(csv_files_list)))
print('DIRTY : {}'.format(len(empty_csv_files)))
ocred_csv_files = glob.glob(os.path.join(config.OCR_CSV_DIR, '*.csv'))
print('OCRed : {}'.format(len(ocred_csv_files)))
pblms_list = [csv_file for csv_file in ocred_csv_files
                       if os.path.getsize(csv_file) == 0]
print('Pblms : {} ({})'.format(len(pblms_list),
                               100*len(pblms_list)/len(csv_files_list))